In [2]:
import os, sys
import django
sys.path.append('../') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps
from django.forms import forms
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [3]:
subject_identifier = 'B142-040991259-8'

onsch_fu = OnScheduleCohortCFU.objects.get(subject_identifier=subject_identifier)

In [5]:
onsch = OnScheduleCohortCFUSeq.objects.get(subject_identifier=subject_identifier)

<OnScheduleCohortCFUSeq: B142-040991259-8 2024-12-10>

In [7]:
from datetime import datetime
import pytz

tz = pytz.timezone('Africa/Gaborone')
dt = datetime(2024, 12, 5, 10, 42)
dt = dt.astimezone(tz)

datetime.datetime(2024, 12, 5, 10, 42, tzinfo=<DstTzInfo 'Africa/Gaborone' CAT+2:00:00 STD>)

In [8]:
onsch.onschedule_datetime = dt
onsch.save()
ssh = SubjectScheduleHistory.objects.get(
    subject_identifier=onsch.subject_identifier, schedule_name=onsch.schedule_name)
ssh.onschedule_datetime = dt
ssh.save()

In [9]:
from edc_visit_schedule.site_visit_schedules import site_visit_schedules
_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model=ssh.onschedule_model, name=ssh.schedule_name)
schedule.put_on_schedule(
    subject_identifier=onsch.subject_identifier,
    onschedule_datetime=onsch.onschedule_datetime,
    schedule_name=onsch.schedule_name,
    base_appt_datetime=onsch.onschedule_datetime)

In [23]:
from flourish_caregiver.models import CaregiverOffSchedule

onsch_quart = OnScheduleCohortCQuarterly.objects.get(
    subject_identifier=subject_identifier, schedule_name='c_quarterly1_schedule1')
CaregiverOffSchedule.objects.create(
    subject_identifier=subject_identifier,
    schedule_name=onsch_quart.schedule_name,
    offschedule_datetime=onsch_fu.onschedule_datetime)

<CaregiverOffSchedule: B142-040991259-8 2024-12-10>

In [10]:
csh = CohortSchedules.objects.get(schedule_type='followup', cohort_name='cohort_c', child_count=1)

In [11]:
_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model=csh.onschedule_model, name=csh.schedule_name)
schedule.put_on_schedule(
    subject_identifier=onsch_fu.subject_identifier,
    onschedule_datetime=onsch_fu.onschedule_datetime,
    schedule_name=csh.schedule_name,
    base_appt_datetime=onsch_fu.onschedule_datetime)

In [18]:
new_onsch = OnScheduleCohortCFU.objects.get(subject_identifier=onsch_fu.subject_identifier,
                                            schedule_name=csh.schedule_name)
new_onsch.child_subject_identifier='B142-040991259-8-10'
new_onsch.save()

In [19]:
from edc_appointment.models import Appointment

visit = MaternalVisit.objects.get(
    subject_identifier=subject_identifier, schedule_name=onsch_fu.schedule_name)
visit_appt = visit.appointment
new_appt = Appointment.objects.get(subject_identifier=subject_identifier,
                                   schedule_name=new_onsch.schedule_name)
new_appt.appt_datetime = visit_appt.appt_datetime
new_appt.appt_type = visit_appt.appt_type
new_appt.appt_status = visit_appt.appt_status
new_appt.appt_reason = visit_appt.appt_reason
new_appt.comment = visit_appt.comment
new_appt.save()
visit.appointment = new_appt
visit.schedule_name = new_appt.schedule_name
visit.visit_schedule_name = new_appt.visit_schedule_name
visit.save()

In [21]:
old_appt = Appointment.objects.get(
    subject_identifier=subject_identifier, schedule_name=onsch_fu.schedule_name)
ssh = SubjectScheduleHistory.objects.get(
    subject_identifier=subject_identifier, schedule_name=onsch_fu.schedule_name)
ssh.delete()
old_appt.delete()
onsch_fu.delete()

(1, {'flourish_caregiver.OnScheduleCohortCFU': 1})